# Caso 14: Data Quality y Error Handling

**Objetivo**: Aprender a manejar datos corruptos, validar calidad y configurar alertas.

## 📋 Contenido

1. Generar datos con errores
2. Leer con manejo de bad records
3. Aplicar validaciones de negocio
4. Quarantine de datos inválidos
5. Reportes de calidad

---

## Realidad en Producción

SIEMPRE hay:
- Registros corruptos (JSON malformado)
- Nulls inesperados
- Datos fuera de rango
- Duplicados
- Schema mismatches

In [ ]:
# Setup
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Caso 14: Data Quality") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print("✅ Spark Session creado")
print(f"   Version: {spark.version}")

## Paso 1: Generar Datos con Errores

Vamos a simular datos reales con problemas comunes.

In [ ]:
import json
import os

# Crear directorio
os.makedirs("/tmp/test_data", exist_ok=True)

# Generar datos buenos y malos
data = []

# Buenos registros (900)
for i in range(1, 901):
    data.append(json.dumps({
        "customer_id": i,
        "name": f"Customer {i}",
        "email": f"customer{i}@example.com",
        "age": 25 + (i % 50),
        "phone": f"+1-555-{i:04d}"
    }))

# Registros con problemas (100)
# Email inválido (30)
for i in range(901, 931):
    data.append(json.dumps({
        "customer_id": i,
        "name": f"Customer {i}",
        "email": "invalid-email",  # ❌ No tiene @
        "age": 30,
        "phone": f"+1-555-{i:04d}"
    }))

# Age inválido (20)
for i in range(931, 951):
    data.append(json.dumps({
        "customer_id": i,
        "name": f"Customer {i}",
        "email": f"customer{i}@example.com",
        "age": -5,  # ❌ Edad negativa
        "phone": f"+1-555-{i:04d}"
    }))

# Null customer_id (20)
for i in range(951, 971):
    data.append(json.dumps({
        "customer_id": None,  # ❌ ID nulo
        "name": f"Customer {i}",
        "email": f"customer{i}@example.com",
        "age": 30,
        "phone": f"+1-555-{i:04d}"
    }))

# JSON malformado (30)
for i in range(971, 1001):
    data.append('{"customer_id": ' + str(i) + ', "name": "Bad')  # ❌ JSON incompleto

# Escribir archivo
with open("/tmp/test_data/customers.json", "w") as f:
    f.write("\n".join(data))

print(f"✅ Generados 1000 registros:")
print(f"   • 900 buenos")
print(f"   • 30 con email inválido")
print(f"   • 20 con age negativo")
print(f"   • 20 con customer_id NULL")
print(f"   • 30 con JSON malformado")

## Paso 2: Leer con PERMISSIVE Mode

Spark puede manejar registros corruptos automáticamente.

In [ ]:
# Leer con manejo de bad records
print("📖 Leyendo datos con PERMISSIVE mode...")

raw_df = spark.read \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "_corrupt_record") \
    .json("/tmp/test_data/customers.json")

print(f"   Total registros leídos: {raw_df.count()}")

# Separar buenos y malos
good_records = raw_df.filter(col("_corrupt_record").isNull()).drop("_corrupt_record")
bad_records = raw_df.filter(col("_corrupt_record").isNotNull())

print(f"\n📊 Separación automática:")
print(f"   • Registros buenos: {good_records.count()}")
print(f"   • Registros corruptos: {bad_records.count()}")

print("\n❌ Ejemplos de registros corruptos:")
bad_records.select("_corrupt_record").show(3, truncate=False)

## Paso 3: Aplicar Validaciones de Negocio

Ahora validamos reglas de negocio en registros que no están corruptos.

In [ ]:
# Aplicar validaciones de negocio
print("✅ Aplicando validaciones de negocio...")

validated = good_records.withColumn(
    "quality_status",
    when(col("customer_id").isNull(), lit("NULL_ID"))
    .when(~col("email").contains("@"), lit("INVALID_EMAIL"))
    .when((col("age") < 0) | (col("age") > 120), lit("INVALID_AGE"))
    .when(col("phone").isNull(), lit("NULL_PHONE"))
    .otherwise(lit("VALID"))
)

# Estadísticas de calidad
print("\n📊 Resultados de validación:")
quality_stats = validated.groupBy("quality_status").count().orderBy(desc("count"))
quality_stats.show()

# Calcular porcentajes
total = validated.count()
for row in quality_stats.collect():
    status = row["quality_status"]
    count = row["count"]
    pct = (count / total) * 100
    symbol = "✅" if status == "VALID" else "❌"
    print(f"   {symbol} {status}: {count:,} ({pct:.1f}%)")

## Paso 4: Separar Datos Válidos e Inválidos

In [ ]:
# Separar válidos e inválidos
valid_records = validated.filter(col("quality_status") == "VALID").drop("quality_status")
invalid_records = validated.filter(col("quality_status") != "VALID")

print(f"📦 Datos limpios: {valid_records.count():,} registros")
print(f"⚠️  Datos problemáticos: {invalid_records.count():,} registros")

print("\n✅ Muestra de datos VÁLIDOS:")
valid_records.show(5)

print("\n❌ Muestra de datos INVÁLIDOS:")
invalid_records.select("customer_id", "name", "email", "age", "quality_status").show(10)

## Paso 5: Quarantine - Guardar Datos Problemáticos

Los datos inválidos se guardan para análisis posterior.

In [ ]:
# Guardar en quarantine con metadata
print("📦 Guardando registros inválidos en quarantine...")

invalid_with_metadata = invalid_records \
    .withColumn("quarantined_at", current_timestamp()) \
    .withColumn("source_file", lit("customers.json")) \
    .withColumn("reason", col("quality_status"))

invalid_with_metadata.write \
    .mode("overwrite") \
    .partitionBy("reason") \
    .parquet("/tmp/quarantine/customers")

print(f"   ✅ {invalid_with_metadata.count():,} registros en quarantine")

# Verificar quarantine por razón
print("\n📊 Distribución de problemas en quarantine:")
spark.read.parquet("/tmp/quarantine/customers") \
    .groupBy("reason") \
    .count() \
    .orderBy(desc("count")) \
    .show()

## Paso 6: Guardar Datos Limpios

Los datos válidos van a la tabla limpia para procesamiento.

In [ ]:
# Guardar datos limpios
print("💾 Guardando datos limpios...")

valid_records.write \
    .mode("overwrite") \
    .parquet("/tmp/clean_data/customers")

print(f"   ✅ {valid_records.count():,} registros guardados")

# Verificar
clean_data = spark.read.parquet("/tmp/clean_data/customers")
print(f"\n📊 Datos limpios verificados: {clean_data.count():,} registros")
clean_data.show(5)

## Paso 7: Reporte de Calidad

Generar reporte completo de data quality.

In [ ]:
# Reporte de calidad completo
total_input = 1000
corrupt_records_count = bad_records.count()
invalid_records_count = invalid_records.count()
valid_records_count = valid_records.count()

quality_score = (valid_records_count / total_input) * 100

print("=" * 70)
print("📊 REPORTE DE DATA QUALITY")
print("=" * 70)
print(f"\n📥 INPUT:")
print(f"   Total registros: {total_input:,}")

print(f"\n❌ PROBLEMAS DETECTADOS:")
print(f"   • JSON corruptos: {corrupt_records_count:,} ({(corrupt_records_count/total_input)*100:.1f}%)")
print(f"   • Validación fallida: {invalid_records_count:,} ({(invalid_records_count/total_input)*100:.1f}%)")
print(f"   • Total problemáticos: {corrupt_records_count + invalid_records_count:,}")

print(f"\n✅ DATOS LIMPIOS:")
print(f"   • Registros válidos: {valid_records_count:,} ({quality_score:.1f}%)")

print(f"\n📊 QUALITY SCORE: {quality_score:.1f}%")

if quality_score >= 95:
    print("   🟢 EXCELENTE - Calidad alta")
elif quality_score >= 90:
    print("   🟡 ACEPTABLE - Revisar problemas")
else:
    print("   🔴 CRÍTICO - Requiere atención inmediata")

print("\n📋 ACCIONES:")
print(f"   • Datos limpios → /tmp/clean_data/customers")
print(f"   • Quarantine → /tmp/quarantine/customers")
print(f"   • Revisar registros en quarantine y corregir upstream")
print("=" * 70)

## 🎯 Conclusiones

### ✅ Lo que Aprendimos

1. **PERMISSIVE mode** captura registros corruptos automáticamente
2. **Validaciones de negocio** con expresiones condicionales
3. **Quarantine tables** para datos problemáticos
4. **Particionamiento por razón** de fallo
5. **Reportes de calidad** con métricas claras

### 💡 Best Practices

- ✅ **Nunca fallar el pipeline** por bad data
- ✅ **Separar datos limpios de problemáticos**
- ✅ **Alertar** cuando calidad cae bajo umbral
- ✅ **Metadata en quarantine** (timestamp, source, reason)
- ✅ **Revisar quarantine regularmente** y corregir upstream

### 🚨 En Producción

```python
# Alerting automático
if quality_score < 90:
    send_alert("Data quality critical: {}%".format(quality_score))
```

### 🔗 Próximos Pasos

- Ver **Caso 13**: CDC - Validar datos CDC
- Ver **Caso 15**: Streaming - Quality en tiempo real
- Implementar con **Great Expectations** para validaciones avanzadas